<a href="https://colab.research.google.com/github/szdziedzic/intent-classification-for-polish-language/blob/main/colab_experiments/herbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.9 MB/s eta 0:00:00


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.8 MB/s eta 0:00:00


In [3]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=e96ffe01819124f0d515f8015c4b0ce923b3bb08e4be376441ad1150edb6058f
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [74]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader, Subset
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [67]:
MASSIVE_INTENTS = ['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock',
            'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe',
            'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up',
            'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup',
            'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query',
            'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query',
            'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim',
            'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down',
            'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff',
            'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music',
            'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set',
            'play_game', 'alarm_remove', 'lists_remove', 'transport_taxi', 'recommendation_movies',
            'iot_coffee', 'music_query', 'play_podcasts', 'lists_query']

train_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='train')
val_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='validation')
test_dataset = load_dataset("AmazonScience/massive", "pl-PL", split='test')

In [65]:
train_dataset[0]

{'id': '1',
 'locale': 'pl-PL',
 'partition': 'train',
 'scenario': 16,
 'intent': 48,
 'utt': 'obudź mnie o dziewiątej rano w piątek',
 'annot_utt': 'obudź mnie o [time : dziewiątej rano] w [date : piątek]',
 'worker_id': '9',
 'slot_method': {'slot': ['time', 'date'],
  'method': ['translation', 'translation']},
 'judgments': {'worker_id': ['21', '11', '5'],
  'intent_score': [1, 1, 1],
  'slots_score': [1, 1, 1],
  'grammar_score': [4, 4, 4],
  'spelling_score': [2, 2, 2],
  'language_identification': ['target', 'target', 'target']}}

In [66]:
model_names = {
    "herbert-klej-cased-v1": {
        "tokenizer": "allegro/herbert-klej-cased-tokenizer-v1",
        "model": "allegro/herbert-klej-cased-v1",
    },
    "herbert-base-cased": {
        "tokenizer": "allegro/herbert-base-cased",
        "model": "allegro/herbert-base-cased",
    },
    "herbert-large-cased": {
        "tokenizer": "allegro/herbert-large-cased",
        "model": "allegro/herbert-large-cased",
    },
}

tokenizer = AutoTokenizer.from_pretrained(model_names["herbert-base-cased"]["tokenizer"])
model = AutoModel.from_pretrained(model_names["herbert-base-cased"]["model"])

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.sso.sso_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [71]:
class MASSIVEDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        example = self.dataset[index]
        return example["utt"], example["intent"]

train_dataloader = DataLoader(Subset(MASSIVEDataset(train_dataset), range(100)), batch_size=32, shuffle=True)
val_dataloader = DataLoader(Subset(MASSIVEDataset(val_dataset), range(50)), batch_size=32, shuffle=True)
test_dataloader = DataLoader(Subset(MASSIVEDataset(test_dataset), range(50)), batch_size=32, shuffle=True)

In [75]:
class MASSIVEIntentClassifier(nn.Module):
  def __init__(self):
        super().__init__()
        self.linear = nn.Linear(model.config.hidden_size, len(MASSIVE_INTENTS))

  def forward(self, x):
      x = self.linear(x)
      return F.softmax(x)

In [77]:
def tokenize_batch(batch):
    return tokenizer.batch_encode_plus(
        batch,
        padding="longest",
        add_special_tokens=True,
        return_tensors="pt",
    )


In [78]:
number_of_epochs = 50

intent_clf = MASSIVEIntentClassifier()
intent_clf = intent_clf.cuda()
loss_fn = nn.CrossEntropyLoss()

opt = torch.optim.Adam(
  intent_clf.parameters(),
  lr=3e-4,
)
epoch_progress = tqdm(list(range(number_of_epochs)))

history = []
for i in epoch_progress:
    train_loss = 0
    y_train_predicted = []
    y_train_true = []
    intent_clf.train()
    for iteration, (X_train, y_train) in enumerate(train_dataloader):
        y_train = y_train.cuda()
        opt.zero_grad()
        output = model(
            **tokenize_batch(X_train)
        )
        pooler_output = output.pooler_output
        pooler_output = pooler_output.cuda()
        y_pred = intent_clf(pooler_output)
        loss = loss_fn(y_pred, y_train)
        loss.backward()
        opt.step()
        train_loss += loss.item()
        y_train_predicted.extend(torch.argmax(y_pred, dim=-1).cpu().numpy())
        y_train_true.extend(y_train.cpu().numpy())


    val_loss = 0
    y_predicted = []
    y_true = []

    intent_clf.eval()
    with torch.no_grad():
        for iteration, (X_val, y_val) in enumerate(val_dataloader):
            y_val = y_val.cuda()
            output = model(
                **tokenize_batch(X_val)
            )
            pooler_output = output.pooler_output
            pooler_output = pooler_output.cuda()
            y_pred = intent_clf(pooler_output)
            loss = loss_fn(y_pred, y_val)
            val_loss += loss.item()
            y_true.extend(y_val.cpu().numpy())
            y_predicted.extend(torch.argmax(y_pred, dim=-1).cpu().numpy())
    train_acc = accuracy_score(y_train_true, y_train_predicted)
    val_acc = accuracy_score(y_true, y_predicted)
    epoch_progress.set_description(f'#Epoch: {i}, train loss: {train_loss:.2f}, train_acc: {train_acc:.2f}, val loss: {val_loss:.2f}, val_acc: {val_acc:.2f}')
    history.append({"e": i, "train_acc": train_acc, "val_acc": val_acc})

  0%|          | 0/50 [00:00<?, ?it/s]<ipython-input-75-77ad92b467f8>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
#Epoch: 0, train loss: 16.37, train_acc: 0.00, val loss: 8.18, val_acc: 0.00:   2%|▏         | 1/50 [00:15<12:21, 15.13s/it]<ipython-input-75-77ad92b467f8>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
#Epoch: 1, train loss: 16.34, train_acc: 0.08, val loss: 8.16, val_acc: 0.00:   4%|▍         | 2/50 [00:28<11:18, 14.15s/it]<ipython-input-75-77ad92b467f8>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)
#Epoch: 2, train loss: 16.30, train_acc: 0.09, val loss: 8.15, val_acc: 0.00:   6%|▌         | 3/50 [00:42<10:53, 13.91s/it]<ipython-input-75-77ad92b467f8>:8: UserWarning: Implicit

KeyboardInterrupt: ignored